In [22]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import MinMaxScaler, RobustScaler

from DataAugmentation import loadAndPrepareAugmentedData, loadAndPrepareOriginalData, trainTestSplit

In [2]:
def color_hist(input_image, nbins=32):
    ch1 = np.histogram(input_image[:,:,0], bins = nbins, range = (0, 256))[0] # [0] is because we need only the histogram, not bins edges
    ch2 = np.histogram(input_image[:,:,1], bins = nbins, range = (0, 256))[0]
    ch3 = np.histogram(input_image[:,:,2], bins = nbins, range = (0, 256))[0]
    return np.hstack((ch1, ch2, ch3))

In [3]:
original_images, original_labels = loadAndPrepareOriginalData()

p = np.random.permutation(len(original_images))
original_images = original_images[p]
original_labels = original_labels[p]

In [4]:
images, labels = loadAndPrepareAugmentedData()

p = np.random.permutation(len(images))
images = images[p]
labels = labels[p]

In [5]:
images = np.array([color_hist(img) for img in images])

In [16]:
original_images = np.array([color_hist(img) for img in original_images])

In [6]:
images.shape

(799932, 96)

In [7]:
model = SVC(gamma=0.001, verbose=1)

In [8]:
labels = [l[0] for l in labels]

In [20]:
original_labels = [l[0] for l in original_labels]

In [23]:
len = 20000

x_train = images[:len]
y_train = labels[:len]

scaling = MinMaxScaler(feature_range=(-1, 1)).fit(x_train)
x_train = scaling.transform(x_train)
original_images = scaling.transform(original_images)

model.fit(x_train, y_train)

[LibSVM]

SVC(gamma=0.001, verbose=1)

In [24]:
model.score(original_images, original_labels)

0.4739652870493992